# PCA + K-means clustering 결과를 이용하여 SVD 학습시키기

In [7]:
!pip install surprise

     |████████████████████████████████| 11.8MB 261kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670922 sha256=df023971a4bf99324b42726644e1a2500a6421ce34abbacb77e061c635358a95
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [16]:
from surprise import Dataset ,Reader, accuracy, SVD, dump
import pandas as pd
import numpy as np
from itertools import chain
from sklearn.utils import shuffle
import pickle

In [9]:
# train data
tr = pd.read_json("/content/drive/My Drive/recommendation sys/0802/train.json") ; tr['id'] = tr.id.map(str) # surprise에서 inner id와 raw id 사이에 헷갈림 방지

# train playlist를 이용해서 곡 별 태그(new_tags)와 cluster id( PCA + K-means 결과)를 추가한 song_meta
song_meta = pd.read_pickle("/content/drive/My Drive/recommendation sys/song_meta_0802.pkl")
pca = pd.read_pickle("/content/drive/My Drive/recommendation sys/추천프로젝트/0801_new_pca.pkl")

In [10]:
tags = pd.unique(list(chain.from_iterable(song_meta.new_tags)))

sid_to_cid = dict(zip(song_meta.id.map(str), song_meta.new_song_id.map(str)))
n_clusters = 1000 # k-means cluster 수

tag_to_tid = {tags[i]:str(n_clusters + i) for i in range(len(tags))}
tid_to_tag = {tid:tag for tag,tid in tag_to_tid.items()}

ply_to_title = {tr.id[i]:tr.plylst_title[i] for i in range(len(tr))}

In [11]:
tr['cid'] = tr.songs.map(lambda x:[sid_to_cid[str(i)] for i in x])
tr['tid'] = tr.tags.map(lambda x:[tag_to_tid[str(i)] for i in x if x in tags])
tr['clusters_tags'] = tr.cid + tr.tid
tr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


,tags,id,plylst_title,songs,like_cnt,updt_date,cid,tid,clusters_tags
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,"[509, 10, 783, 550, 346, 346, 604, 745, 357, 7...",[1019],"[509, 10, 783, 550, 346, 346, 604, 745, 357, 7..."
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[954, 525, 804, 834, 386, 452, 273, 0, 0, 145,...",[],"[954, 525, 804, 834, 386, 452, 273, 0, 0, 145,..."
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[523, 370, 935, 578, 493, 630, 322, 253, 293, ...",[],"[523, 370, 935, 578, 493, 630, 322, 253, 293, ..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[451, 668, 165, 55, 396, 833, 53, 782, 510, 13...",[],"[451, 668, 165, 55, 396, 833, 53, 782, 510, 13..."
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,"[954, 236, 90, 736, 797, 156, 278, 648, 736, 2...",[1058],"[954, 236, 90, 736, 797, 156, 278, 648, 736, 2..."
...,...,...,...,...,...,...,...,...,...
115066,"[록메탈, 밴드사운드, 록, 락메탈, 메탈, 락, extreme]",120325,METAL E'SM #2,"[429629, 441511, 612106, 516359, 691768, 38714...",3,2020-04-17 04:31:11.000,"[989, 829, 829, 829, 738, 585, 604, 284, 284, ...",[],"[989, 829, 829, 829, 738, 585, 604, 284, 284, ..."
115067,[일렉],106976,빠른 리스너를 위한 따끈따끈한 최신 인기 EDM 모음!,"[321330, 216057, 534472, 240306, 331098, 23288...",13,2015-12-24 17:23:19.000,"[20, 677, 517, 242, 20, 20, 517, 517, 575, 517...",[1014],"[20, 677, 517, 242, 20, 20, 517, 517, 575, 517..."
115068,"[담시, 가족, 눈물, 그리움, 주인공, 나의_이야기, 사랑, 친구]",11343,#1. 눈물이 앞을 가리는 나의_이야기,"[50512, 249024, 250608, 371171, 229942, 694943...",4,2019-08-16 20:59:22.000,"[164, 108, 482, 937, 856, 937, 937, 235, 133, ...",[],"[164, 108, 482, 937, 856, 937, 937, 235, 133, ..."
115069,"[잔잔한, 버스, 퇴근버스, Pop, 풍경, 퇴근길]",131982,퇴근 버스에서 편히 들으면서 하루를 마무리하기에 좋은 POP,"[533534, 608114, 343608, 417140, 609009, 30217...",4,2019-10-25 23:40:42.000,"[731, 197, 758, 533, 731, 282, 103, 731, 731, ...",[],"[731, 197, 758, 533, 731, 282, 103, 731, 731, ..."


# surprise에 들어갈 input data 만들기
* rating은 빈도를 이용.
* play list에 i번째 song(또는 tag)가 1번 들어갔다면 점수는 1. 5번 들어갔다면 점수는 5

In [27]:
def flatten_series(nested_series, index):
    repeated_index = np.repeat(index, nested_series.map(len))
    return pd.DataFrame(chain.from_iterable(nested_series), index = repeated_index)

In [28]:
cl_tg = flatten_series(tr.clusters_tags, tr.id).reset_index().rename(columns = {0:'iid'})
cl_tg = cl_tg.groupby(cl_tg.columns.tolist()).size().reset_index().rename(columns = {0:'rating'})
cl_tg

,id,iid,rating
0,1,338,2
1,1,415,1
2,1,470,2
3,1000,21,1
4,1000,265,1
...,...,...,...
2553110,99999,424,1
2553111,99999,625,1
2553112,99999,680,1
2553113,99999,857,1


# train (SVD)
* `customedSVD` : user id, item id, rating으로 이루어진 df와 latent factor 수, epoch 수를 입력하면 SVD 모델과 train_object를 return합니다.
* `customedAccuracy` : SVD 모델과 train_object를 입력하면 예측 결과와 rmse를 return합니다,
* `pred_table` : 예측 결과를 dataframe 형태로 return합니다. 해당 item을 가진 play list수 또는 해당 play list가 가진 item수와 error를 확인할 수 있습니다.

In [48]:
def customedSVD(df, n_factors, n_epochs):

    print("input type :",type(df.iloc[0,0]),type(df.iloc[0,1]))
    df.iloc[:,0] = df.iloc[:,0].map(str)
    df.iloc[:,1] = df.iloc[:,1].map(str)
    print("raw id type",type(df.iloc[0,0]),type(df.iloc[0,1]))
    display(df.head())

    reader = Reader(rating_scale=(min(df.iloc[:,2]), max(df.iloc[:,2])))

    data = Dataset.load_from_df(df, reader)
    train_object = data.build_full_trainset()

    reg_rate = 0.02
    algo = SVD(verbose = True, n_epochs = n_epochs, n_factors = n_factors, reg_bu = reg_rate , reg_bi = reg_rate , reg_pu = reg_rate, reg_qi = reg_rate, random_state = 152152)
    algo.fit(train_object)

    return algo, train_object

def customedAccuracy(algo, train_object):
    val = train_object.build_testset()
    predictions = algo.test(val)
    rmse = accuracy.rmse(predictions)

    return predictions, rmse

def pred_table(train_obj, predictions, mapping = True):

    def get_Iu(uid): # 해당 play list의 song/tag 수
        try:
            return len(train_obj.ur[train_obj.to_inner_uid(uid)])
        except ValueError:  # user was not part of the trainset
            return 0
    
    def get_Ui(iid): # 해당 song/tag를 가진 play list 수
        try:
            return len(train_obj.ir[train_obj.to_inner_iid(iid)])
        except ValueError:
            return 0

    df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])    
    df['#of_tag_given_ply'] = df.uid.apply(get_Iu)
    df['#of_ply_given_tag'] = df.iid.apply(get_Ui)
    df['err'] = abs(df.est - df.rui)

    if mapping:
        df['uid'] = df['uid'].map(ply_to_title)

    return df

In [ ]:
# latent factor 512개, epoch 50 (RMSE: 0.2009)

rating_series = cl_tg.rating.tolist()
algo, train_obj = customedSVD(cl_tg, 512, 50)
predictions, rmse = customedAccuracy(algo, train_obj) ;print("rmse :",rmse) ; print(predictions[:5])

input type : <class 'str'> <class 'str'>
raw id type <class 'str'> <class 'str'>
     id  iid rating
0  1000   21      1
1  1000  265      1
2  1000  352      1
3  1000  362      1
4  1000  364      1
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processin

In [ ]:
# # prediction시 이용하기 위해 pickle로 저장

# with open("/content/drive/My Drive/recommendation sys/algo_0802.pkl","wb") as f:
#     pickle.dump(algo, f, pickle.HIGHEST_PROTOCOL)

# with open("/content/drive/My Drive/recommendation sys/train_obj_0802.pkl","wb") as f:
#     pickle.dump(train_obj, f, pickle.HIGHEST_PROTOCOL)

# prediction 결과 확인

In [18]:
res = pred_table(train_obj, predictions).sort_values('iid')
res

,uid,iid,rui,est,details,#of_tag_given_ply,#of_ply_given_tag,err
312949,봄봄봄봄,0,1.0,1.123694,{'was_impossible': False},20,1454,0.123694
2258524,들으면 동화같은 일이 일어날것같은 음악들,0,1.0,1.084838,{'was_impossible': False},21,1454,0.084838
107316,헐! 알고보니 무늬만 인디 52,0,2.0,1.931349,{'was_impossible': False},14,1454,0.068651
1591567,오늘의 추천 가수 - 장기하와얼굴들,0,4.0,3.619749,{'was_impossible': False},30,1454,0.380251
1231302,Lets Rock Festival 2016 (최종 라인업),0,1.0,1.404407,{'was_impossible': False},64,1454,0.404407
...,...,...,...,...,...,...,...,...
1437939,"옛날 냄새, 옛날 감성, 옛날 복고풍의 신나는 노래들~",999,1.0,1.000000,{'was_impossible': False},95,389,0.000000
488510,멋짐 폭발✴️ New Jack Swing Flavah‼☺️,999,1.0,1.118932,{'was_impossible': False},27,389,0.118932
1994476,조용한 아침을 깨우는 deep& chill out mix,999,1.0,1.104407,{'was_impossible': False},21,389,0.104407
2533308,"황폐한 내 마음, 대신 울어 줄래요?",999,1.0,1.000000,{'was_impossible': False},14,389,0.000000


In [22]:
res.sort_values('err').head(15)

,uid,iid,rui,est,details,#of_tag_given_ply,#of_ply_given_tag,err
1896027,래퍼들의 이야기는 우리들 이야기가 된다..,736,1.0,1.0,{'was_impossible': False},47,11039,0.0
1228199,외로움에....사무칠때~,278,1.0,1.0,{'was_impossible': False},20,11773,0.0
2149905,비 오는 날 듣고 싶은 노래들,170,1.0,1.0,{'was_impossible': False},17,3301,0.0
2069269,그냥 지나칠 수 없는 독특한 매력을 가진 취향저격 곡들!,625,1.0,1.0,{'was_impossible': False},12,9829,0.0
1373048,잔잔하게 비 오는 날 듣기 좋은 곡,278,1.0,1.0,{'was_impossible': False},22,11773,0.0
1890383,샘플링으로 완성된 일렉트로닉 음악들,477,1.0,1.0,{'was_impossible': False},13,1607,0.0
2091297,MBC FM4U 라디오 14.12.14 (일) : 바람이 분다,170,1.0,1.0,{'was_impossible': False},57,3301,0.0
453836,힙합이 낯설은 모든 리스너를 위한 셀렉션,846,1.0,1.0,{'was_impossible': False},14,19500,0.0
1007695,한국 시티팝 모음집,625,1.0,1.0,{'was_impossible': False},12,9829,0.0
688602,새벽 감성과 로맨스와 이별,625,1.0,1.0,{'was_impossible': False},28,9829,0.0


In [21]:
res.sort_values('err', ascending = False).head(15)

,uid,iid,rui,est,details,#of_tag_given_ply,#of_ply_given_tag,err
2544512,"팝 음악의 역사, 로큰롤의 황제 엘비스 프레슬리!",815,119.0,131.034471,{'was_impossible': False},5,2104,12.034471
306378,이선희 노래 전곡,220,75.0,66.104677,{'was_impossible': False},55,4441,8.895323
35490,그것에 알고 싶다! 12화(휘성 편),263,63.0,56.220243,{'was_impossible': False},43,3092,6.779757
1176370,드라이브 하면서 따라부르는 남자들의 추억의 락발라드 1990-2010,525,53.0,46.781030,{'was_impossible': False},29,6259,6.218970
533565,다시 듣고싶은 복면가왕 노래,166,29.0,22.888879,{'was_impossible': False},74,4396,6.111121
118079,해질녁 푸른 바다가 보이는 곳에서 듣고 싶은 음악 #2,218,63.0,57.094389,{'was_impossible': False},41,2175,5.905611
1680772,ahahah,590,43.0,37.444007,{'was_impossible': False},46,5181,5.555993
530185,좋아하는,14,46.0,40.449095,{'was_impossible': False},89,3324,5.550905
423711,이소라 노래 모음~~,220,29.0,23.538052,{'was_impossible': False},59,4441,5.461948
1385510,oooooo,590,47.0,41.604397,{'was_impossible': False},45,5181,5.395603
